# PureStorage REST API Demonstration for Active Cluster - Carlos Carrero

#### We need a REST API Token - Let's get one for us to use for this demonstration.  Here are the steps:

<ol>
    <li>__Navigate to your FlashArray IP: https://xx.xx.xx.xx</li>
    <li>__Navigate to the users section:  System -> Users -> API Tokens__</li>
    <li>__Create an API token if you don't already have one__</li>
</ol>



<table>
    <tr><td> __Hostname__ </td><td>lon-m20-a (one array of AC configuration at London)</td></tr>
    <tr><td> __IP__ </td><td>https://xx.xx.xx.xx</td></tr>
    <tr><td> __Token__ </td><td>bxxxxxxxb-4xxd-xxxx-xxxx-0xxxxxxxx8db</td></tr>
</table>



In [65]:
import json
import pandas
import requests
import time

import urllib3
# We're using a version that warns us about insecure requests, and we don't want to see that noise.
urllib3.disable_warnings(urllib3.exceptions.InsecureRequestWarning)
%matplotlib inline

# ENTER THE IP ADDRESS OF YOUR ARRAY AND THE TOKEN
IP='10.222.10.10'
token='bxxxxxxxb-4xxd-xxxx-xxxx-0xxxxxxxx8db'

## Overview of PureStorage FlashArray Rest API basics

#### FlashStache leverages a REST API wrapper called purestorage - it's the official python API for the purestorage REST service.

#### You can find more information about this module at 
* __[Documentation Overview](http://pure-storage-python-rest-client.readthedocs.io/en/latest/)__
* __[Installation Instructions](http://pure-storage-python-rest-client.readthedocs.io/en/latest/installation.html)__
* __[purestorage python API glossary](http://pure-storage-python-rest-client.readthedocs.io/en/latest/api.html)__
* __[rest-client](https://github.com/purestorage/rest-client/blob/master/purestorage/purestorage.py)__


#### To import all of the api, the call is simple:

In [66]:
import purestorage

#### Now we don't have to manage the sessions or make explicit calls, we can use the API to handle the busywork for us.  For example, instead of our rest call for a session creation and storing that, we can use the array object

In [67]:
array = purestorage.FlashArray(IP, api_token=token)
array

#### Now instead of a rest call, we can use the basic .get() to pull the same information we got from the REST response

In [68]:
array_info = array.get()
array_info

{u'array_name': u'lon-m20-a',
 u'id': u'1279d4fc-9e68-4912-bbe9-0308d2e4b79f',
 u'revision': u'201805172031+fbb0082',
 u'version': u'5.1.1'}

#### How about our volume information?

In [69]:
volume_info = array.list_volumes()
volume_info[1]

{u'created': u'2018-01-03T11:42:53Z',
 u'name': u'z-citypod::z-lon-esx65-heartbeat1',
 u'serial': u'1279D4FC9E68491200011010',
 u'size': 53687091200,
 u'source': None}

In [74]:
for eachvol in volume_info:
    volume_name = eachvol["name"]
    print "  Volumen: {}".format(volume_name)


  Volumen: z-lon-win2016-a-localvol
  Volumen: z-citypod::z-lon-esx65-heartbeat1
  Volumen: z-citypod::z-lon-esx65-heartbeat2
  Volumen: z-citypod::z-lon-esx65-activecluster
  Volumen: z-citypod::z-lon-win2016-oracle
  Volumen: z-citypod::z-lon-win2016-win-witness
  Volumen: z-citypod::lon-win2016-a-sql-vol01
  Volumen: z-citypod::lon-win2016-fileserver
  Volumen: z-citypod::z-sql-failover-clst
  Volumen: z-citypod::z-win2016-sql-data
  Volumen: z-citypod::z-zfs-vol1
  Volumen: z-citypod::z-zfs-vol2
  Volumen: z-citypod::z-zfs-solaris-vol1
  Volumen: z-citypod::z-win2016-sql-data02
  Volumen: z-win2016-sql-log02
  Volumen: z-citypod::z-win2016-sql-log02
  Volumen: z-lon-pcopy-vol01
  Volumen: z-lon-pcopy-vol02
  Volumen: eg-test--win2016-async
  Volumen: z-gl-01
  Volumen: z-gl-02
  Volumen: z-gl-03
  Volumen: z-citypod::z-sol-test
  Volumen: fred01
  Volumen: z-citypod::fred02
  Volumen: z-LMH-pod::z-LMH-AC-DS
  Volumen: fredvol01
  Volumen: duncanvol
  Volumen: LMH133214234123
  Volu

# Active Cluster: Now we are going to create a new volume and we are going to strech it into the two arrays mappint it to one ESX server in each site

In [82]:
#Create a new volume
vol = array.create_volume('vol1-cc', '10M')
vol

{u'created': u'2018-06-03T17:58:49Z',
 u'name': u'vol1-cc',
 u'serial': u'1279D4FC9E6849120009D760',
 u'size': 10485760,
 u'source': None}

In [28]:
## Take a snapshot of the volume - just to proof the API

In [83]:
snap1 = array.create_snapshot('vol1-cc')
snap1

{u'created': u'2018-06-03T17:58:59Z',
 u'name': u'vol1-cc.1',
 u'serial': u'1279D4FC9E6849120009D761',
 u'size': 10485760,
 u'source': u'vol1-cc'}

In [84]:
vol = array.get_volume('vol1-cc')
vol

{u'created': u'2018-06-03T17:58:49Z',
 u'name': u'vol1-cc',
 u'serial': u'1279D4FC9E6849120009D760',
 u'size': 10485760,
 u'source': None}

In [85]:
# List available pods
pods = array.list_pods()
for p in pods:
    print ("Pod: {}".format(p["name"]))
    


Pod: z-citypod
Pod: z-LMH-pod
Pod: nickpod
Pod: LMHTESTPOD
Pod: testpod
Pod: spie


In [ ]:
# Create test pod if it does not exists - rememberr the 6 Pod limit (also check there are no deleted pods as they also count)
podtest = array.create_pod('testpod')
podtest

In [87]:
# Strech previous volume into the pod so it will be mirrored across the two arrays

array.move_volume('vol1-cc', 'testpod')


{u'created': u'2018-06-03T17:58:49Z',
 u'name': u'testpod::vol1-cc',
 u'serial': u'1279D4FC9E6849120009D760',
 u'size': 10485760,
 u'source': None}

In [88]:
# Map the volume into each of the ESX hosts
# List the hosts first

hosts= array.list_hosts()
for h in hosts:
    print ("Host: {}".format(h["name"]))

Host: z-lon-win2016-b
Host: z-lon-esx65-b
Host: z-lon-esx65-a
Host: z-lon-win2016-a
Host: z-solaris
Host: z-ca-mark
Host: markmunich
Host: z-lon-pcopy
Host: cfrdemo
Host: headithost
Host: purestoragehost
Host: maples


In [90]:
# Do the mapping

array.connect_host("z-lon-esx65-b", "testpod::vol1-cc")
array.connect_host("z-lon-esx65-a", "testpod::vol1-cc")


{u'lun': 5, u'name': u'z-lon-esx65-a', u'vol': u'testpod::vol1-cc'}

In [ ]:
# Here I will continue with APIs for vCenter to creater a Data Store on the new Volume,
# creater some VMs that use that DS and run some testing



# CLEANUP - Let´s leave thing cleans after the demo

In [91]:
array.disconnect_host("z-lon-esx65-b", "testpod::vol1-cc")
array.disconnect_host("z-lon-esx65-a", "testpod::vol1-cc")

{u'name': u'z-lon-esx65-a', u'vol': u'testpod::vol1-cc'}

In [92]:
array.destroy_volume('testpod::vol1-cc')

{u'name': u'testpod::vol1-cc'}

In [94]:
array.eradicate_volume('testpod::vol1-cc')

{u'name': u'testpod::vol1-cc'}